# Add all necessary libraries!

In [15]:
"""
@author: Sadman Sakib
"""

import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, confusion_matrix
from sklearn.model_selection import StratifiedKFold
from PIL import Image
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import warnings
import os
import shutil
from PIL import ImageFile
warnings.simplefilter('error', Image.DecompressionBombWarning)
ImageFile.LOAD_TRUNCATED_IMAGES = True
Image.MAX_IMAGE_PIXELS = 1000000000
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)


C:\Users\sadma\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\client\session.py:1766: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [16]:
# Make it False if do not intend to use Google Colab and want to train in local machine!!
google_colab_flag = False

# For training in Google Colab
if(google_colab_flag):
    from google.colab import drive
    drive.mount('/content/drive')
    !ls
    import sys
    # This is the path to where in google drive the code is stored!
    root_path = '/content/drive/My Drive/Practice/Stratified-k-fold-cross-validation-Image-classification-keras/'
    sys.path.append(root_path)

# For local training
else:
    root_path = ''

In [17]:
dataset_folder_name = os.path.join(root_path, 'DS')
MODEL_FILENAME = root_path+"model_cv.h5"
source_files = []
class_labels = ['circles', 'squares', 'triangles']
X = []
Y = []

img_rows, img_cols = 100, 100  # input image dimensions
train_path = os.path.join(dataset_folder_name, 'train')
validation_path = os.path.join(dataset_folder_name, 'validation')
test_path = os.path.join(dataset_folder_name, 'test')


In [18]:
def transfer_between_folders(source, dest, split_rate):
    global source_files
    source_files = os.listdir(source)
    if(len(source_files) != 0):
        transfer_file_numbers = int(len(source_files)*split_rate)
        transfer_index = random.sample(
            range(0, len(source_files)), transfer_file_numbers)
        for each_index in transfer_index:
            shutil.move(os.path.join(source, str(source_files[each_index])), os.path.join(dest, str(source_files[each_index])) )

    else:
        print("No file moved. Source empty!")


def transfer_all_class_between_folders(source, dest, split_rate):
    for label in class_labels:
        transfer_between_folders(os.path.join(dataset_folder_name, source, label),
                               os.path.join(dataset_folder_name, dest, label),
                               split_rate)                           


def my_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    f1Score = f1_score(y_true, y_pred, average='weighted')
    print("Accuracy  : {}".format(accuracy))
    print("Precision : {}".format(precision))
    print("f1Score : {}".format(f1Score))
    cm = confusion_matrix(y_true, y_pred)
    print(cm)
    return accuracy, precision, f1Score


# First, check if test folder is empty or not, if not transfer all existing files to train.

In [19]:
transfer_all_class_between_folders('test', 'train', 1.0)


# Now, split some part of train data into the test folders.

In [20]:
transfer_all_class_between_folders('train', 'test', 0.20)


In [21]:
def prepare_name_with_labels(folder_name):
    source_files = os.listdir(os.path.join(dataset_folder_name, 'train', folder_name))
    for val in source_files:
        X.append(val)
        for i in range(len(class_labels)):
            if(folder_name == class_labels[i]):
                Y.append(i)


In [22]:
# Organize file names and class labels in X and Y variables
for i in range(len(class_labels)):
    prepare_name_with_labels(class_labels[i])


In [23]:
X = np.asarray(X)
Y = np.asarray(Y)


In [24]:
batch_size = 512
epoch = 100
num_of_channels = 3
number_of_class_labels = len(class_labels)

In [25]:
"""
    Note that, this model structure is a very basic one. 
    To achieve better performance, you should change the model structure and hyperparameters according to your needs and data. 
    So, optimize the structure of the model!
"""

def get_model():
    activation_function = 'relu'

    model = Sequential()
    model.add(Conv2D(64, (3, 3), padding='same',
                     activation=activation_function, input_shape=(img_rows, img_cols, num_of_channels)))
    model.add(Conv2D(64, (3, 3), activation=activation_function))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(32, (3, 3), padding='same',
                     activation=activation_function))
    model.add(Conv2D(32, (3, 3), activation=activation_function))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(16, (3, 3), padding='same',
                     activation=activation_function))
    model.add(Conv2D(16, (3, 3), activation=activation_function))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(64, activation=activation_function))
    model.add(Dropout(0.1))
    model.add(Dense(32, activation=activation_function))
    model.add(Dropout(0.1))
    model.add(Dense(16, activation=activation_function))
    model.add(Dropout(0.1))
    model.add(Dense(number_of_class_labels, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model


model = get_model()


# **Stratified K-Fold Cross validation**

In [26]:
# ===============Stratified K-Fold======================
skf = StratifiedKFold(n_splits=3, shuffle=True)
skf.get_n_splits(X, Y)
fold_num = 0
for train_index, val_index in skf.split(X, Y):
    # First cut all images from validation to train (if any exists)
    transfer_all_class_between_folders('validation', 'train', 1.0)
    fold_num += 1
    print("Results for fold", fold_num)
    X_train, X_val = X[train_index], X[val_index]
    Y_train, Y_val = Y[train_index], Y[val_index]
    # Move validation images of this fold from train folder to the validation folder
    for each_index in range(len(X_val)):
        class_label = ''
        for i in range(len(class_labels)):
            if(Y_val[each_index] == i):
                class_label = class_labels[i]
        # Then, copy the validation images to the validation folder
        shutil.move(os.path.join(dataset_folder_name, 'train', class_label, X_val[each_index]),
                    os.path.join(dataset_folder_name, 'validation', class_label, X_val[each_index]))

    train_datagen = ImageDataGenerator(
        rescale=1./255,
        zoom_range=0.20,
        fill_mode="nearest")
    validation_datagen = ImageDataGenerator(rescale=1./255)
    test_datagen = ImageDataGenerator(rescale=1./255)

    # Start ImageClassification Model
    train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical',
        subset='training')

    validation_generator = validation_datagen.flow_from_directory(
        validation_path,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode=None,  # only data, no labels
        shuffle=False)

    # fit model
    history = model.fit(train_generator,
                        epochs=epoch)

    predictions = model.predict(validation_generator, verbose=1)
    y_predictions = np.argmax(predictions, axis=1)
    true_classes = validation_generator.classes
    # evaluate validation performance
    print("***Performance on Validation data***")
    val_acc, val_prec, val_fScore = my_metrics(true_classes, y_predictions)


Results for fold 1
Found 191 images belonging to 3 classes.
Found 64 images belonging to 3 classes.
Epoch 1/100
1/1 [==============================] - 1s 1s/step - loss: 1.1010 - accuracy: 0.3089
Epoch 2/100
1/1 [==============================] - 0s 473ms/step - loss: 1.1026 - accuracy: 0.2932
Epoch 3/100
1/1 [==============================] - 0s 472ms/step - loss: 1.1028 - accuracy: 0.2827
Epoch 4/100
1/1 [==============================] - 0s 468ms/step - loss: 1.0985 - accuracy: 0.2984
Epoch 5/100
1/1 [==============================] - 0s 446ms/step - loss: 1.1002 - accuracy: 0.3194
Epoch 6/100
1/1 [==============================] - 0s 455ms/step - loss: 1.0993 - accuracy: 0.3298
Epoch 7/100
1/1 [==============================] - 0s 497ms/step - loss: 1.0968 - accuracy: 0.3455
Epoch 8/100
1/1 [==============================] - 0s 458ms/step - loss: 1.1022 - accuracy: 0.3246
Epoch 9/100
1/1 [==============================] - 0s 467ms/step - loss: 1.0992 - accuracy: 0.3246
Epoch 10/100

# Test phase

In [27]:
print("==============TEST RESULTS============")
test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode=None,
    shuffle=False 
)
predictions = model.predict(test_generator, verbose=1)
y_predictions = np.argmax(predictions, axis=1)
true_classes = test_generator.classes

test_acc, test_prec, test_fScore = my_metrics(true_classes, y_predictions)
model.save(MODEL_FILENAME)


==============TEST RESULTS============
Found 45 images belonging to 3 classes.
1/1 [==============================] - 0s 47ms/step
Accuracy  : 0.8444444444444444
Precision : 0.8438492063492063
f1Score : 0.8435545668026202
[[13  0  2]
 [ 0 14  1]
 [ 3  1 11]]
